In [1]:
%load_ext autoreload
%autoreload 2
%cd ..

/Users/williamsoma/finn-deals


In [2]:
import pandas as pd

In [3]:
df = pd.read_csv("data/raw/gitar_listings_260219.csv")
df["timestamp"] = pd.to_datetime(df["timestamp"], unit="ms")
df["time_since_posted"] = (pd.Timestamp.now() - df["timestamp"])
df.info()
with pd.option_context('display.max_columns', None):
    display(df)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8050 entries, 0 to 8049
Data columns (total 46 columns):
 #   Column                   Non-Null Count  Dtype          
---  ------                   --------------  -----          
 0   type                     8050 non-null   object         
 1   id                       8050 non-null   int64          
 2   main_search_key          8050 non-null   object         
 3   heading                  8050 non-null   object         
 4   location                 8050 non-null   object         
 5   flags                    8050 non-null   object         
 6   timestamp                8050 non-null   datetime64[ns] 
 7   ad_type                  8050 non-null   int64          
 8   canonical_url            8050 non-null   object         
 9   distance                 8050 non-null   int64          
 10  trade_type               8050 non-null   object         
 11  image_urls               8050 non-null   object         
 12  ad_id               

,type,id,main_search_key,heading,location,flags,timestamp,ad_type,canonical_url,distance,trade_type,image_urls,ad_id,is_promoted,image_url,image_path,image_height,image_width,image_aspect_ratio,coordinates_lat,coordinates_lon,coordinates_accuracy,price_amount,price_currency_code,price_price_unit,brand,organisation_name,extra_brand,label_ids,label_texts,is_private,is_retailer,has_shipping,has_buy_now,num_images,timestamp_dt,page_current,page_last,total_match_count,total_group_count,search_query,fashion_hub_collections,extra_clothing_size,consoles_type,extra_consoles_type,time_since_posted
0,bap,451514767,SEARCH_ID_BAP_ALL,LTD 87’ Eclipse,Oslo,"private, shipping_exists, buy_now",2026-02-17 13:50:37.445,67,https://www.finn.no/recommerce/forsale/item/45...,0,Til salgs,['https://images.finncdn.no/dynamic/default/it...,451514767,True,https://images.finncdn.no/dynamic/default/item...,item/451514767/a9b6ab0d-6d4f-41cb-89e3-96153fd...,4032.0,3024.0,0.750000,59.91260,10.77691,5,10500,NOK,kr,NaN,NaN,NaN,"fiks_ferdig, buy_now, private","Fiks ferdig, Kjøp nå, Privat",True,False,True,True,2,2026-02-17 13:50:37.445000+00:00,1,50,8050,8050,gitar,NaN,NaN,NaN,NaN,5 days 08:14:13.884379
1,bap,450523928,SEARCH_ID_BAP_ALL,Klassisk gitar CD-album,Oslo,private,2026-02-10 20:01:15.000,67,https://www.finn.no/recommerce/forsale/item/45...,0,Gis bort,['https://images.finncdn.no/dynamic/default/it...,450523928,False,https://images.finncdn.no/dynamic/default/item...,item/450523928/8c14bc94-df26-4500-afff-a198dbb...,3024.0,4032.0,1.333333,59.91740,10.81920,5,0,NOK,kr,NaN,NaN,NaN,private,Privat,True,False,False,False,4,2026-02-10 20:01:15+00:00,1,50,8050,8050,gitar,NaN,NaN,NaN,NaN,12 days 02:03:36.329379
2,bap,450063721,SEARCH_ID_BAP_ALL,Phil Capone Lær deg å spille gitar lærebok med CD,Fyllingsdalen,private,2026-02-08 07:11:39.000,67,https://www.finn.no/recommerce/forsale/item/45...,0,Gis bort,['https://images.finncdn.no/dynamic/default/it...,450063721,False,https://images.finncdn.no/dynamic/default/item...,item/450063721/a75f901f-e1cf-4d41-8227-94395ba...,4032.0,3024.0,0.750000,60.34225,5.28754,5,0,NOK,kr,NaN,NaN,NaN,private,Privat,True,False,False,False,3,2026-02-08 07:11:39+00:00,1,50,8050,8050,gitar,NaN,NaN,NaN,NaN,14 days 14:53:12.329379
3,bap,310963832,SEARCH_ID_BAP_ALL,Burns Cobra Stratocaster type Elektrisk gitar ...,Kongsberg,private,2026-01-24 13:45:53.000,67,https://www.finn.no/recommerce/forsale/item/31...,0,Gis bort,['https://images.finncdn.no/dynamic/default/5b...,310963832,False,https://images.finncdn.no/dynamic/default/5b/5...,5b/5b273f4e-092d-49e1-b230-5e1c2d08c6a1,4640.0,3472.0,0.748276,59.68042,9.63562,5,0,NOK,kr,NaN,NaN,NaN,private,Privat,True,False,False,False,1,2026-01-24 13:45:53+00:00,1,50,8050,8050,gitar,NaN,NaN,NaN,NaN,29 days 08:18:58.329379
4,bap,419829368,SEARCH_ID_BAP_ALL,Dimebag gitarer,Flatåsen,private,2026-01-25 14:05:08.000,67,https://www.finn.no/recommerce/forsale/item/41...,0,Ønskes kjøpt,['https://images.finncdn.no/dynamic/default/da...,419829368,False,https://images.finncdn.no/dynamic/default/da/d...,da/da4a5500-d8ed-44b5-ae86-c66ccd52c5f3,960.0,720.0,0.750000,63.37723,10.33834,5,0,NOK,kr,NaN,NaN,NaN,private,Privat,True,False,False,False,6,2026-01-25 14:05:08+00:00,1,50,8050,8050,gitar,NaN,NaN,NaN,NaN,28 days 07:59:43.329379
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8045,bap,445481385,SEARCH_ID_BAP_ALL,Marshall JVM 410C 100W 4-kanaler,Heimdal,private,2026-01-10 19:44:28.000,67,https://www.finn.no/recommerce/forsale/item/44...,0,Til salgs,['https://images.finncdn.no/dynamic/default/it...,445481385,False,https://images.finncdn.no/dynamic/default/item...,item/445481385/1083ae6e-63a5-4eb3-be56-ceb9b8e...,1235.0,1179.0,0.954656,63.35390,10.31051,5,150000,NOK,kr,NaN,NaN,NaN,private,Privat,True,False,False,False,5,2026-01-10 19:44:28+00:00,5,5,259,259,gitar,NaN,NaN,NaN,NaN,43 days 0

In [ ]:
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder
import numpy as np

# ── Explicitly pick useful features (not auto-discover) ──
# Rationale for every inclusion/exclusion documented below.

# Binary flags — all directly describe the listing
binary_cols = ["is_private", "is_retailer", "has_shipping", "has_buy_now"]
# Dropped: is_promoted (ad-spend signal, not item quality)

# Numeric — actual item/listing attributes
numeric_cols = [
    "coordinates_lat",    # geographic signal (EDA: MI=0.27)
    "coordinates_lon",    # geographic signal (EDA: MI=0.28)
    "num_images",         # seller effort proxy (EDA: r=+0.43)
    "distance",           # distance to buyer / centre
]
# Dropped:
#   ad_type               — unclear integer, likely internal
#   image_height/width/aspect_ratio — thumbnail metadata, not item property
#   coordinates_accuracy  — precision of pin, not useful
#   page_current/last, total_match_count/group_count — search pagination metadata

# Categorical — only cols with signal for guitar prices
categorical_cols = [
    "trade_type",         # Til salgs / Gis bort / Ønskes kjøpt (3 levels)
    "brand",              # 24 guitar-relevant brands
]
# Dropped:
#   type                     — constant (1 unique value)
#   extra_brand              — duplicate of brand
#   organisation_name        — only 14 orgs, overlaps with is_retailer
#   fashion_hub_collections  — irrelevant for guitars
#   extra_clothing_size      — irrelevant for guitars
#   consoles_type            — irrelevant for guitars
#   extra_consoles_type      — irrelevant for guitars
#   location                 — 747 unique strings → use lat/lon instead

# --- Binary features ---
df_binary = df[binary_cols].astype(int)

# --- Numeric features (min-max scaled, NaN → 0) ---
num_scaler = MinMaxScaler()
df_numeric = pd.DataFrame(
    num_scaler.fit_transform(df[numeric_cols].fillna(0)),
    columns=[f"num_{c}" for c in numeric_cols],
    index=df.index,
)

# --- Categorical features (one-hot, low cardinality) ---
df_cat_filled = df[categorical_cols].fillna("missing")
ohe = OneHotEncoder(sparse_output=False, handle_unknown="ignore")
cat_encoded = ohe.fit_transform(df_cat_filled)
cat_feature_names = ohe.get_feature_names_out(categorical_cols)
df_categorical = pd.DataFrame(cat_encoded, columns=cat_feature_names, index=df.index)

# --- Combine ---
X_tabular = pd.concat([df_binary, df_numeric, df_categorical], axis=1)

print(f"=== Feature summary ===")
print(f"Binary features:      {len(binary_cols):>3}  {binary_cols}")
print(f"Numeric features:     {len(numeric_cols):>3}  {numeric_cols}")
print(f"Categorical features: {len(cat_feature_names):>3}  (from {len(categorical_cols)} columns)")
print(f"{'─'*50}")
print(f"Total tabular dims:   {X_tabular.shape[1]:>3}  (was 871)")
print(f"Rows:                 {X_tabular.shape[0]:>5}")
display(X_tabular.head())

In [4]:
from finn_deals.modeling.utils import WordPieceTokenizer, Log1pMinMaxScaler
from torch.nn.utils.rnn import pad_sequence
import numpy as np
from typing import List, Iterable, Dict
from sklearn.model_selection import train_test_split
import torch

tokenizer = WordPieceTokenizer()

# Build vocab from dataframe titles
vocab = " ".join(df["heading"].str.lower().str.replace(r"[^a-z0-9]+", " ", regex=True).str.strip().tolist()).split()

tokenizer.fit(vocab)
X = pd.DataFrame()
X["header"] = tokenizer(df["heading"])

def min_max_scaling(data):
    return (data - np.min(data)) / (np.max(data) -  np.min(data))

X["timestamp"] = min_max_scaling(df["timestamp"].astype(int).values)
X["time_since_posted"] = min_max_scaling(df["time_since_posted"].astype(int).values)

scaler = Log1pMinMaxScaler()
y = scaler.fit_transform(df["price_amount"].values)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# X_train = torch.nested.nested_tensor(X_train, layout=torch.jagged)
# X_test = torch.nested.nested_tensor(X_test, layout=torch.jagged)
X_train_header = pad_sequence(
    [torch.tensor(x, dtype=torch.long) for x in X_train["header"]],
    batch_first=True,
    padding_value=tokenizer.pad_token_id,
)
X_train_time = torch.tensor(X_train[["timestamp", "time_since_posted"]].values, dtype=torch.float32)

X_test_header = pad_sequence(
    [torch.tensor(x, dtype=torch.long) for x in X_test["header"]],
    batch_first=True,
    padding_value=tokenizer.pad_token_id,
)
X_test_time = torch.tensor(X_test[["timestamp", "time_since_posted"]].values, dtype=torch.float32)
y_train = torch.tensor(y_train, dtype=torch.float32)
y_test = torch.tensor(y_test, dtype=torch.float32)

print(pd.Series(vocab).value_counts())

gitar     1514
med        533
og         430
fender     429
til        372
          ... 
marx         1
ricard       1
laxer        1
1114         1
1946         1
Name: count, Length: 10144, dtype: int64


In [5]:
from finn_deals.modeling.nn import SentimentModel, SentimentModelConfig
cfg = SentimentModelConfig(
    vocab_size = len(tokenizer),
    seq_max_len=100,
    positional_encoding_type="sinusoidal",
    embedding_dim=32,
    padding_idx = tokenizer.pad_token_id,
    mlp_hidden_layers=[64, 64],
)
model = SentimentModel(cfg)
print(cfg)

TypeError: non-default argument 'num_features' follows default argument 'num_time_features'

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import math

device = torch.device("cuda" if torch.cuda.is_available() else ("mps" if torch.backends.mps.is_available() else "cpu"))
print(f"Using device: {device}")

model = SentimentModel(cfg).to(device)

# Move header (token ids) and time tensors to device
train_header = X_train_header.to(device)
train_time   = X_train_time.to(device)    # already (B, 2)
test_header  = X_test_header.to(device)
test_time    = X_test_time.to(device)     # already (B, 2)

y_tr = y_train.to(device).float()
y_te = y_test.to(device).float()

criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=1e-3)

num_iterations = 5000
patience = 50
best_val_loss = float('inf')
steps_without_improvement = 0
best_state_dict = None

for it in range(1, num_iterations + 1):

    # ---- train step ----
    model.train()
    optimizer.zero_grad()
    pred = model(train_header, train_time)
    loss = criterion(pred, y_tr)
    loss.backward()
    optimizer.step()

    # ---- validation / test loss ----
    model.eval()
    with torch.no_grad():
        test_pred = model(test_header, test_time)
        test_loss = criterion(test_pred, y_te)

    # ---- early stopping ----
    if test_loss.item() < best_val_loss:
        best_val_loss = test_loss.item()
        steps_without_improvement = 0
        best_state_dict = {k: v.clone() for k, v in model.state_dict().items()}
    else:
        steps_without_improvement += 1

    # ---- logging ----
    if it % 10 == 0:
        rmse = math.sqrt(loss.item())
        test_rmse = math.sqrt(test_loss.item())
        print(
            f"Iter {it:05d} | Train RMSE: {rmse:.4f} | Test RMSE: {test_rmse:.4f}"
        )

    if steps_without_improvement >= patience:
        print(f"Early stopping at iteration {it}")
        break

# restore best model
model.load_state_dict(best_state_dict)
print(f"\nBest Test RMSE: {math.sqrt(best_val_loss):.4f}")

# final predictions
pred = model(test_header, test_time).cpu().detach().numpy()

In [ ]:
import matplotlib.pyplot as plt

plt.hist(y_test.cpu().detach().numpy() - pred, bins=50)
plt.yscale("log")
plt.show()


plt.hist(scaler.untransform(y_test.cpu().detach().numpy()) - scaler.untransform(pred), bins=50)
plt.yscale("log")
plt.show()



In [ ]:
from sklearn.metrics import r2_score, mean_absolute_error

print(f"Nominal R²: {r2_score(scaler.untransform(y_test.cpu().detach().numpy()), scaler.untransform(pred)):.2%}")
print(f"Log1p R²: {r2_score(y_test.cpu().detach().numpy(), pred):.2%}")

print(f"Nominal MAE: {mean_absolute_error(scaler.untransform(y_test.cpu().detach().numpy()), scaler.untransform(pred)):,.2f}")

Nominal R²: 2.78%
Log1p R²: 34.25%
Nominal MAE: 4,330.39

In [ ]:
# Quick check — did the new feature cell actually run?
print(f"binary_cols = {binary_cols}")
print(f"numeric_cols = {numeric_cols}")
print(f"categorical_cols = {categorical_cols}")
print(f"X_tabular.shape = {X_tabular.shape}")